# Lab | Inferential statistics - T-test & P-value

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

## One tailed t-test

In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file `files_for_lab/machine.txt`.

Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

**H0**: new_machine_speed <= old_machine_speed<br>
**H1**: new_machine_speed > old_machine_speed

**Significance level**: 0.95<br>
**Confidence level**: 0.05

In [2]:
data = pd.read_csv('files_for_lab/machine.txt', encoding='UTF-16', sep='\t')
data

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [3]:
data.columns = ['new_machine', 'old_machine']

In [4]:
pval = st.ttest_ind(data['new_machine'], data['old_machine'], equal_var=False)[1]
pval

0.00324224946631798

In [5]:
print("Accept H0" if pval < 1-0.05 else "Reject H0")

Accept H0


##### Accept H0, therefore the new machine is FASTER than the old one

<br>

## Matched Pairs Test
In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment your result.

**H0**: defense == attack<br>
**H1**: defense != attack

**Significance level**: 0.95<br>
**Confidence level**: 0.05/2  (two-sided)

In [6]:
data = pd.read_csv('files_for_lab/pokemon.csv')
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [7]:
pval = st.ttest_rel(data['Attack'], data['Defense'])[1]
pval

1.7140303479358558e-05

In [8]:
print("Accept H0" if 0.025 < pval < 1-0.025 else "Reject H0")

Reject H0


##### We reject H0, therefore there's a significant difference between attack and defense score

<br>

## ANOVA - Part 1

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on:
    - Null hypothesis
    - Alternate hypothesis
    - Level of significance
    - Test statistic
    - P-value
    - F table

### Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data `anova_lab_data.xlsx` file in the `files_for_lab` folder  

- State the null hypothesis
- State the alternate hypothesis
- What is the significance level
- What are the degrees of freedom of model, error terms, and total DoF

Data was collected randomly and provided to you in the table as shown: [link to the image - Data](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.05/7.05-lab_data.png)


**H0**: m1 = m2 = ... = mN<br>
**H1**: m1 != m2 != ... != mN

**Significance level**: 0.95<br>
**Confidence level**: 0.05

**DoF_between**: K-1 = 2<br>
**DoF_within**: N-K = 2<br>
**DoF_total**: N-1 = 4


In [20]:
data = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
data

15

In [10]:
data.columns = ['power', 'etching_rate']

In [11]:
data['power_count'] = data.groupby('power').cumcount()
data

,power,etching_rate,power_count
0,160 W,5.43,0
1,180 W,6.24,0
2,200 W,8.79,0
3,160 W,5.71,1
4,180 W,6.71,1
5,200 W,9.20,1
6,160 W,6.22,2
7,180 W,5.98,2
8,200 W,7.90,2
9,160 W,6.01,3


In [12]:
data_pivot = data.pivot(index='power_count', columns='power', values='etching_rate')
data_pivot.columns = ['power_'+str(x).replace(' ', '_') for x in data_pivot.columns.values]
data_pivot

,power_160_W,power_180_W,power_200_W
power_count,,,
0,5.43,6.24,8.79
1,5.71,6.71,9.20
2,6.22,5.98,7.90
3,6.01,5.66,8.15
4,5.59,6.60,7.55


In [13]:
data_pivot.mean()

power_160_W    5.792
power_180_W    6.238
power_200_W    8.318
dtype: float64

In [14]:
st.f_oneway(data_pivot.power_160_W,
            data_pivot.power_180_W,
            data_pivot.power_200_W)

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)

In [15]:
print("Accept H0" if pval >= 0.05 else "Reject H0")

Reject H0


##### Since our p-value is smaller than our confidence level, we reject H0.
There is a difference in the mean etching rate for different levels of power.